In [40]:
import h5py
h5f = h5py.File('zwei_populationen.h5', 'r')
P_0_10000 = h5f['P_0_10000']
print(P_0_10000)
P_0_10000 = h5f['P_0_10000'][:]

<HDF5 group "/P_0_10000" (2 members)>


AttributeError: 'slice' object has no attribute 'encode'